In [3]:
pip install scikit-image

Note: you may need to restart the kernel to use updated packages.


In [6]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
import os
from tqdm import tqdm

In [9]:
# Apply the proposed decomposition and enhancement algorithm
def enhance_image(image):
    # Convert the image to HSV space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Normalize the V channel
    norm_V = hsv[:,:,2] / 255.0

    # Apply the proposed decomposition
    L = np.exp(-1 * np.log(norm_V + 0.001))
    R = norm_V / L

    # Adjust the illumination
    L_adjusted = cv2.normalize(L, None, alpha=0.5, beta=400, norm_type=cv2.NORM_MINMAX)

    # Generate the enhanced V channel image
    enhanced_V = L_adjusted * R
    enhanced_V = cv2.normalize(enhanced_V, None, alpha=0, beta=1500, norm_type=cv2.NORM_MINMAX)
    enhanced_V = enhanced_V.astype(np.uint8)

    # Merge the enhanced V channel with the H and S channels to get the enhanced HSV image
    enhanced_hsv = cv2.merge((hsv[:,:,0], hsv[:,:,1], enhanced_V))

    # Convert the enhanced HSV image to RGB space
    enhanced_image = cv2.cvtColor(enhanced_hsv, cv2.COLOR_HSV2BGR)
  
    return enhanced_image


In [10]:
def psnr(img1, img2):
    # Calculate MSE (Mean Squared Error)
    mse = np.mean((img1 - img2) ** 2)

    # If images are identical, PSNR is infinity
    if mse == 0:
        return float('inf')

    # Calculate PSNR using formula: PSNR = 20 * log10(MAX_I) - 10 * log10(MSE)
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel) - 10 * np.log10(mse)
    return psnr

In [1]:
# Path to the folder containing images
folder_path = r"./input"

In [12]:
# Get a list of all the image files in the folder
image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg', '.bmp'))]
total_images = len(image_files)

In [13]:
# Use tqdm to create a progress bar with message and percentage
with tqdm(total=total_images, desc="Processing images") as pbar:
    # Loop through all the image files
    lst1=[]
    for i, file_name in enumerate(image_files):
        # Load the image
        img_path = os.path.join(folder_path, file_name)
        image = cv2.imread(img_path)
        image=cv2.resize(image,(612,812))
        # Apply the enhancement algorithm
        enhanced_image = enhance_image(image)

        #Saving the image on local drive
        output_path=r'./output'
        out_path = os.path.join(output_path,file_name)
        #Uncomment this to stack two images(Original image and enhanced image)
        #result=np.hstack((image,enhanced_image))
        cv2.imwrite(out_path,enhanced_image)

        #  Calculate PSNR and print the result
        psnr_value = psnr(image,enhanced_image)
        psnr_value = psnr(image, enhanced_image)
        lst1.append((file_name, psnr_value))

        # Update the progress bar
        pbar.update(1)
        pbar.set_postfix_str("{:.1f}%".format((i+1)/total_images*100))

# Print PSNR values with file names
for file_name, psnr_value in lst1:
    print(f"File: {file_name}, PSNR: {psnr_value}")


Processing images: 100%|██████████| 485/485 [00:52<00:00,  9.18it/s, 100.0%]

File: 10.png, PSNR: 28.332105549310775
File: 100.png, PSNR: 29.527521143008663
File: 101.png, PSNR: 28.376468106665804
File: 102.png, PSNR: 29.007710854956446
File: 103.png, PSNR: 31.325991000737652
File: 104.png, PSNR: 28.365570773770425
File: 105.png, PSNR: 28.10586280258616
File: 106.png, PSNR: 28.009417928909322
File: 107.png, PSNR: 30.547936861794383
File: 109.png, PSNR: 28.232595952297817
File: 110.png, PSNR: 28.19865902098446
File: 112.png, PSNR: 27.582158602948113
File: 113.png, PSNR: 27.939198799449052
File: 114.png, PSNR: 28.606777936449905
File: 115.png, PSNR: 28.386590442256058
File: 116.png, PSNR: 28.39637123630242
File: 117.png, PSNR: 29.39706719667202
File: 118.png, PSNR: 28.47488919776791
File: 119.png, PSNR: 27.62623375634726
File: 12.png, PSNR: 27.975959313673194
File: 120.png, PSNR: 28.00055154538803
File: 121.png, PSNR: 28.66508321951278
File: 122.png, PSNR: 28.054259254620533
File: 123.png, PSNR: 27.555458623508862
File: 124.png, PSNR: 27.679308564829732
File: 125.